##Preprocessing


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vishnukarande","key":"95c389e7e296c20a0f21b0472b1be6ff"}'}

In [ ]:
!mkdir -p ~/.kaggle 

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

100% 721M/723M [00:07<00:00, 137MB/s]
100% 723M/723M [00:07<00:00, 95.8MB/s]


In [ ]:
!unzip jigsaw-unintended-bias-in-toxicity-classification.zip

Archive:  jigsaw-unintended-bias-in-toxicity-classification.zip
  inflating: all_data.csv            
  inflating: identity_individual_annotations.csv  
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: test_private_expanded.csv  
  inflating: test_public_expanded.csv  
  inflating: toxicity_individual_annotations.csv  
  inflating: train.csv               


In [ ]:
Data=pd.read_csv("/content/train.csv")

In [ ]:
Data[['comment_text']].head()

,comment_text
0,"This is so cool. It's like, 'would you want yo..."
1,Thank you!! This would make my life a lot less...
2,This is such an urgent design problem; kudos t...
3,Is this something I'll be able to install on m...
4,haha you guys are a bunch of losers.


In [ ]:
''''#Text preprocessing
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q  
    '''



'\'#Text preprocessing\ndef preprocess(q):\n    \n    q = str(q).lower().strip()\n    \n    # Replace certain special characters with their string equivalents\n    q = q.replace(\'%\', \' percent\')\n    q = q.replace(\'$\', \' dollar \')\n    q = q.replace(\'₹\', \' rupee \')\n    q = q.replace(\'€\', \' euro \')\n    q = q.replace(\'@\', \' at \')\n    \n    # The pattern \'[math]\' appears around 900 times in the whole dataset.\n    q = q.replace(\'[math]\', \'\')\n    \n    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)\n    q = q.replace(\',000,000,000 \', \'b \')\n    q = q.replace(\',000,000 \', \'m \')\n    q = q.replace(\',000 \', \'k \')\n    q = re.sub(r\'([0-9]+)000000000\', r\'\x01b\', q)\n    q = re.sub(r\'([0-9]+)000000\', r\'\x01m\', q)\n    q = re.sub(r\'([0-9]+)000\', r\'\x01k\', q)\n    \n    # Decontracting words\n    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions\n    # https://

This Process Takes a lots of time

In [ ]:
#Data['comment_text']=Data['comment_text'].apply(preprocess)

In [ ]:
for i,j in enumerate(Data['comment_text'].sample(n=5).values):
    print(i,':\n',j,'\n')
    print('*'*100,'\n')

0 :
 I believe it may.  Also Ca Ching Chin is getting involved just to grand stand for his run for gov...Somebody Please Save Hawaii is he is elected! 

**************************************************************************************************** 

1 :
 Well, you know what poor memories this administration has. Sessions completely forgot about his meetings with the Russians. Pence doesn't remember any warnings on Flynn's foreign ties provided to the transition committee he headed. 

And Donnie Jr. keeps forgetting who attended the meeting he claims was about adoption. Seems a new attendee pops up daily, including an eighth attendee - an apparent Russian money launderer, who no doubt was very concerned about finding homes for Russian orphans.

https://thinkprogress.org/trump-jr-meeting-ikakly-kaveladze-russian-money-laundering-scheme-9986f528982a 

**************************************************************************************************** 

2 :
 Ronni:  Guess what?  Trum

#Handling emoji

In [ ]:
 !pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 7.1 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=830a8020bed04df1c27d1e571401ca98c73e6d4d55055cd822eba3383d8f2f36
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [ ]:
import emoji
X='Python is 👍'

In [ ]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
import string
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
# white_list = string.ascii_letters + string.digits + latin_similar + ' '
white_list = string.ascii_letters + latin_similar + ' '
white_list += "'"

In [ ]:
from tqdm import tqdm
jigsaw_chars = build_vocab(list(Data["comment_text"]))
jigsaw_symbols = ''.join([c for c in jigsaw_chars if not c in white_list])
jigsaw_symbols

100%|██████████| 1804874/1804874 [00:57<00:00, 31629.54it/s]


'.,?!-;*"…:\n150—()%263#$97&_8/@＼・ω4+🍕=”“[]^–>\r🐵\\°<😑~\xa0\ue014•≠\t™\uf818\uf04a\xadˈʊɒ😢🐶∞§{}·τα❤️☺ɡ\uf0e0😜😎👊\u200b\u200e😁|عدويهصقأناخلىبمغر😍💖¢→̶`💵❥━┣┫Е┗Ｏ►★👎😀😂\u202a\u202c🔥😄©―🏻💥ᴍʏʀɪᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ✔®\x96\x92●😋👏שלוםבי😱‼£\x81♥エンジ故障➤´\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘¹☕≈÷אעכח♡◐║▬💩′ɔː💯⛽€🚄🏼ஜ۩۞†😖ᴠ🚲‐μ✒➥😟😈═☆ˌ💪🙏🎯◄🌹😇💔½ʻ😡\x7f👌ἐπὶδηλήσειὲκἀίῃἴρξνʃ🙄✬ＳＵＰＥＲＨＩＴ😠\ufeff☻±\u2028😉😤⛺♍🙂µ\u3000تحكسة👮💙فزط😏º🍾🎉¾😞\u2008🏾😅😭👻😥😔😓🏽🎆✓◾🍻🍽🎶🌺🤔😪\x08‑؟🐰🐇🐱🙆．😨⬅🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷℅»ВулканПвоАН🐾🐕❣😆ה⋅🔗¿¬🚽歌舞伎🙈😴🏿🤗🇺🇸♫мυтѕＣＭ⤵🏆🎃β😩█▓▒░\u200a🌠🐟💫💰💎⇒эпрд\x95🖐🙅⛲🍰⭐🤐👆›🙌\u2002💛🙁👀🙊🙉¡₂₃\u2004❧▰ˢᵒʳʸ▔ᴼᴷᴺʷᵗʰᵉᵘ◞▀\x13🚬▂▃▄▅▆▇↙🤓\ue602😵άοόςέγὸ̄תמדףנרךצט😒͝″☹➡«🆕👅👥👄🔄🔤👉👤👶👲🔛🎓φ\uf0b7⅓„✋：\uf04c\x9f\x10成都¥😣⏺̲̅😌🤑́🌏😯ех😲∙‛Ἰᾶὁ💞🚓◇🔔📚✏🏀👐\u202d💤🍇\ue613小土豆🏡▷❔❓⁉❗\u202f👠¶》कर्मा🇹🇼🌸蔡英文🌞˚🎲レクサス😛˙外国人关系）Ссиб💋💀🎄💜🤢َِʿьыгя✨不是。ɑ\x80\x9c\x9d🗑\u2005💃📣👿༼つ◕༽😰ḷЗз▱ц￼🤣卖！温哥华议会下降％你失去所有的钱加拿大坏税骗子🐝¯ツ🎅\x85🍺آإشء−ﬂﬁ🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003₁²🚀🤴ʌʲш¼⁴⁄₄⌠чИОРФДЯМю♭ж✘😝🖑ὐύύ特殊作戦群╪щ💨圆明园ק▶ℐ☭✭🏈😺♪🌍⏏ệ🍔🐮🍁☔🍆🍑🌮🌯☠🤦\u200d♂𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺☃🍸🥂🗽🎇🎊🆘☎🤠👩✈🖒✌✰❆☙🚪天一家⚲\u2006⚭⚆⬭⬯⏖○‣⚓新年∎ℒ▪▙☏⅛✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡

 Overall Observation: 
 1. removing Emojis from data is not good idea. emojis holds lots of sentimental meaning, we have to find the way in which we can convert emojis to its approximate meaning https://stackoverflow.com/questions/57580288/how-to-replace-emoji-to-word-in-a-text

In [ ]:
import emoji
def emojis_to_word(text):
    text = emoji.demojize(text, delimiters=("", ""))
    text = text.replace('_',' ')
    return text

In [ ]:
emojis_to_word('python is 🙄')

'python is face with rolling eyes'

#Convert Into lower case
LOWER=lower

In [ ]:
import contractions
def decontraction(text):
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))   

    expanded_text = ' '.join(expanded_words)
    return expanded_text

In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 7.9 MB/s 
     |████████████████████████████████| 106 kB 63.4 MB/s 


In [ ]:
decontraction("You ain’t see nobody like this before.")

'You are not see nobody like this before.'

In [ ]:
def removing_links(text):
    text = re.sub("(http|https)://[\w\-]+(\.[\w\-]+)+\S*", "", text)
    return text

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
#Lemmatization
''' toy example'''
# import these modules
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print("better :", lemmatizer.lemmatize("better"))

better : better


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
def Lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(str(text))
    return text

In [ ]:
Lemmatization('what a Moe-ron')

'what a Moe-ron'

In [ ]:
#Applying Stemming

#Applying Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [ ]:
stem_words('studies studying cries  cry goods sacas cried ')

'studi studi cri cri good saca cri'

Removing non english word

In [ ]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
import nltk

words = set(nltk.corpus.words.words())

sent = "we plays history day words ansdknak"

result = " ".join(w for w in nltk.wordpunct_tokenize(sent)if w.lower() in words or not w.isalpha())
print('\noriginal:\n',sent,'\n\nafter NLTK :\n',result)


original:
 we plays history day words ansdknak 

after NLTK :
 we history day


In [ ]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
import string
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
# white_list = string.ascii_letters + string.digits + latin_similar + ' '
white_list = string.ascii_letters + latin_similar + ' '
white_list += "'"

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
STOPWORDS = set(stopwords.words('english'))
print(len(STOPWORDS))
STOPWORDS = list(STOPWORDS - {'no','nor','not'})
print(len(STOPWORDS))

179
176


In [ ]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [ ]:
sent = 'we plays history or i did we can  day'
a=remove_stopwords(sent)
a

'plays history day'

In [ ]:
def remove_jigsaw_symbols(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', jigsaw_symbols))

In [ ]:
def remove_jigsaw_symbol_stop_word(text):
    ## Dont change order of calling function 1.remove_jigsaw_symbols and 2.remove_stopwords
    text = remove_jigsaw_symbols(text)
    text = remove_stopwords(text)
    return text


In [ ]:
sent = 'we plays😮 😃 😘¹ history  降你失去所  a did not ,or  有的钱加拿大坏税骗 day  骗good '
a = remove_jigsaw_symbol_stop_word(sent)
a

'plays history not day good'

#Removing http/https links

In [ ]:
temp = 'You assume that they would staff it from external states with a influx of 200,000 people.  That is not necessarily true.What if they recruited 50,000 employees locally?  That is more likely.  Some executive types might move here from elsewhere, but most of the hiring would be here - just like with their warehouse. http://www.denverpost.com/2017/01/23/colorado-amazon-fulfillment-center-aurora/ '
temp

'You assume that they would staff it from external states with a influx of 200,000 people.  That is not necessarily true.What if they recruited 50,000 employees locally?  That is more likely.  Some executive types might move here from elsewhere, but most of the hiring would be here - just like with their warehouse. http://www.denverpost.com/2017/01/23/colorado-amazon-fulfillment-center-aurora/ '

In [ ]:
def removing_links(text):
    text = re.sub("(http|https)://[\w\-]+(\.[\w\-]+)+\S*", "", text)
    return text

In [ ]:
a = removing_links(temp)
a

'You assume that they would staff it from external states with a influx of 200,000 people.  That is not necessarily true.What if they recruited 50,000 employees locally?  That is more likely.  Some executive types might move here from elsewhere, but most of the hiring would be here - just like with their warehouse.  '

removal of Numbers,tabs and newline (\t,\n)

In [ ]:
def remove_num_tab_newline(text):
    text = re.sub(r"[\d\t\n]*", "", text)
    return text

In [ ]:
temp='ABC \n12223 212 sss hey \t  hi\t  hi2 \ajkaalalla'
temp

'ABC \n12223 212 sss hey \t  hi\t  hi2 \x07jkaalalla'

In [ ]:
a = remove_num_tab_newline(temp)
a

'ABC   sss hey   hi  hi \x07jkaalalla'

 removal Punctuations(\t,\n)

In [ ]:
def remove_extra_speces(text):
    text = re.sub(r"[\s\s]+", " ", text)
    return text

In [ ]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
import string
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
# white_list = string.ascii_letters + string.digits + latin_similar + ' '
white_list = string.ascii_letters + latin_similar + ' '
white_list += "'"

In [ ]:
from tqdm import tqdm
# https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part1-eda
jigsaw_chars = build_vocab(list(Data["comment_text"]))
jigsaw_symbols = ''.join([c for c in jigsaw_chars if not c in white_list])
jigsaw_symbols

100%|██████████| 1804874/1804874 [00:49<00:00, 36175.89it/s]


'.,?!-;*"…:\n150—()%263#$97&_8/@＼・ω4+🍕=”“[]^–>\r🐵\\°<😑~\xa0\ue014•≠\t™\uf818\uf04a\xadˈʊɒ😢🐶∞§{}·τα❤️☺ɡ\uf0e0😜😎👊\u200b\u200e😁|عدويهصقأناخلىبمغر😍💖¢→̶`💵❥━┣┫Е┗Ｏ►★👎😀😂\u202a\u202c🔥😄©―🏻💥ᴍʏʀɪᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ✔®\x96\x92●😋👏שלוםבי😱‼£\x81♥エンジ故障➤´\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘¹☕≈÷אעכח♡◐║▬💩′ɔː💯⛽€🚄🏼ஜ۩۞†😖ᴠ🚲‐μ✒➥😟😈═☆ˌ💪🙏🎯◄🌹😇💔½ʻ😡\x7f👌ἐπὶδηλήσειὲκἀίῃἴρξνʃ🙄✬ＳＵＰＥＲＨＩＴ😠\ufeff☻±\u2028😉😤⛺♍🙂µ\u3000تحكسة👮💙فزط😏º🍾🎉¾😞\u2008🏾😅😭👻😥😔😓🏽🎆✓◾🍻🍽🎶🌺🤔😪\x08‑؟🐰🐇🐱🙆．😨⬅🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷℅»ВулканПвоАН🐾🐕❣😆ה⋅🔗¿¬🚽歌舞伎🙈😴🏿🤗🇺🇸♫мυтѕＣＭ⤵🏆🎃β😩█▓▒░\u200a🌠🐟💫💰💎⇒эпрд\x95🖐🙅⛲🍰⭐🤐👆›🙌\u2002💛🙁👀🙊🙉¡₂₃\u2004❧▰ˢᵒʳʸ▔ᴼᴷᴺʷᵗʰᵉᵘ◞▀\x13🚬▂▃▄▅▆▇↙🤓\ue602😵άοόςέγὸ̄תמדףנרךצט😒͝″☹➡«🆕👅👥👄🔄🔤👉👤👶👲🔛🎓φ\uf0b7⅓„✋：\uf04c\x9f\x10成都¥😣⏺̲̅😌🤑́🌏😯ех😲∙‛Ἰᾶὁ💞🚓◇🔔📚✏🏀👐\u202d💤🍇\ue613小土豆🏡▷❔❓⁉❗\u202f👠¶》कर्मा🇹🇼🌸蔡英文🌞˚🎲レクサス😛˙外国人关系）Ссиб💋💀🎄💜🤢َِʿьыгя✨不是。ɑ\x80\x9c\x9d🗑\u2005💃📣👿༼つ◕༽😰ḷЗз▱ц￼🤣卖！温哥华议会下降％你失去所有的钱加拿大坏税骗子🐝¯ツ🎅\x85🍺آإشء−ﬂﬁ🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003₁²🚀🤴ʌʲш¼⁴⁄₄⌠чИОРФДЯМю♭ж✘😝🖑ὐύύ特殊作戦群╪щ💨圆明园ק▶ℐ☭✭🏈😺♪🌍⏏ệ🍔🐮🍁☔🍆🍑🌮🌯☠🤦\u200d♂𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺☃🍸🥂🗽🎇🎊🆘☎🤠👩✈🖒✌✰❆☙🚪天一家⚲\u2006⚭⚆⬭⬯⏖○‣⚓新年∎ℒ▪▙☏⅛✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡

Create Own vocabulary

In [ ]:
## Glove file
from numpy import asarray
# load the whole embedding into memory
embeddings_index = dict()
f = open(r"/content/glove.6B.100d.txt",encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 34219 word vectors.


In [ ]:
def Non_english_word_dropping(text):
    final = []
    text_temp = text.split()
    for i in text_temp:
        if i in embeddings_index:
            final.append(i)
    text = ' '.join(final)
    return text

In [ ]:
sent = 'what a Moe-ron'
result = Non_english_word_dropping(sent)

In [ ]:
sent = 'we plays history  降你失去所 有的钱加拿大坏税骗 day  骗good '
result = Non_english_word_dropping(sent)
print('\noriginal:\n',sent,'\n\nafter NLTK :\n',result)


original:
 we plays history  降你失去所 有的钱加拿大坏税骗 day  骗good  

after NLTK :
 we plays history day


In [ ]:
def preprocessing_DL(Data,submission_data):
    ## Note : dont change preprocessing sequence
    ''' Converting Emojis to word'''
    Data["comment_text_pre"] = Data["comment_text"].progress_apply(lambda text: emojis_to_word(text))
    '''Converting in Lower case'''
    Data["comment_text_pre"] = Data['comment_text_pre'].str.lower()
    '''Decontraction'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: decontraction(text))
    '''Lemmatization'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: Lemmatization(text))
    '''removing_links'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: removing_links(text))
    '''remove_jigsaw_symbol/character_stop_word'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: remove_jigsaw_symbol_stop_word(text))
    '''Non_english_word_dropping'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: Non_english_word_dropping(text))
    '''remove_num_tab_newline'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: remove_num_tab_newline(text))
    '''remove_extra_speces'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: remove_extra_speces(text))
    
    if submission_data == False:
        '''Converting class probability into label'''
        Data['class'] = Data['target'].progress_apply(lambda x: "non-toxic" if x < 0.5 else "toxic")
    return Data

In [ ]:
def preprocessing_ML(Data,submission_data):
    ## Note : dont change preprocessing sequence
    ''' Converting Emojis to word'''
    Data["comment_text_pre"] = Data["comment_text"].progress_apply(lambda text: emojis_to_word(text))
    '''Converting in Lower case'''
    Data["comment_text_pre"] = Data['comment_text_pre'].str.lower()
    '''Decontraction'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: decontraction(text))
    '''removing_links'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: removing_links(text))
    '''remove_jigsaw_symbol/character_stop_word'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: remove_jigsaw_symbol_stop_word(text))
    '''Non_english_word_dropping'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: Non_english_word_dropping(text))
    '''Stemming'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: stem_words(text))
    '''remove_num_tab_newline'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: remove_num_tab_newline(text))
    '''remove_extra_speces'''
    Data["comment_text_pre"] = Data["comment_text_pre"].progress_apply(lambda text: remove_extra_speces(text))
    
    if submission_data == False:
        '''Converting class probability into label'''
        Data['class'] = Data['target'].progress_apply(lambda x: "non-toxic" if x < 0.5 else "toxic")
    return Data

In [ ]:
from tqdm import tqdm
tqdm.pandas()

##Create Small Dataset

Preprocessing the data for ML models



In [ ]:
sample_data = Data.sample(n = 100)
sample_data.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
421424,759216,0.000000,A pause in B.C.'s hot real estate market might...,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,158614,approved,0,0,1,1,0,0.0,0,4
1702633,6209669,0.000000,"It's right wing bologna. Colorado, which is ru...",0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,392493,approved,0,0,0,0,0,0.0,0,4
1022921,5368046,0.000000,"@Osgoodschlotter: That ""dindu"" jab originated ...",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,340993,rejected,0,0,0,0,0,0.0,4,4
187815,470870,0.600000,The Chinese do not make mistakes when it comes...,0.0,0.7,0.100000,0.1,0.0,0.4,0.0,...,145783,approved,0,0,0,0,0,0.6,10,10
757837,5048051,0.833333,Can we please have a heterosexual leader.\n\nW...,0.0,0.0,0.833333,0.0,0.0,0.0,0.0,...,321806,rejected,0,0,0,0,0,0.0,4,6


In [ ]:
Train_Final_pre_text_data_sample_ML= preprocessing_ML(sample_data,submission_data=False)

100%|██████████| 100/100 [00:00<00:00, 74738.13it/s]


In [ ]:
Train_Final_pre_text_data_sample_ML

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,comment_text_pre,class
421424,759216,0.000000,A pause in B.C.'s hot real estate market might...,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,1,1,0,0.0,0,4,paus hot real estat market might good thing mi...,non-toxic
1702633,6209669,0.000000,"It's right wing bologna. Colorado, which is ru...",0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,right wing bologna colorado run big oil ga wor...,non-toxic
1022921,5368046,0.000000,"@Osgoodschlotter: That ""dindu"" jab originated ...",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,4,4,jab origin propaganda machin use tool unwittin...,non-toxic
187815,470870,0.600000,The Chinese do not make mistakes when it comes...,0.0,0.7,0.100000,0.1,0.0,0.4,0.0,...,0,0,0,0,0,0.6,10,10,chines not make mistak come protocol forc obam...,toxic
757837,5048051,0.833333,Can we please have a heterosexual leader.\n\nW...,0.0,0.0,0.833333,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,4,6,pleas heterosexu leader would possibl break no...,toxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825252,5130890,0.000000,I'm all in on this one!,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,one,non-toxic
908050,5230772,0.000000,"Steven said ""Your inevitable inability to prov...",0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,0,1,1,0.0,0,4,steven said inevit inabl provid evid taken adm...,non-toxic
319561,633846,0.000000,"Obviously, your opinion shows your lack of int...",0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,0,0,2,0.0,0,4,obvious opinion show lack intim oper experi on...,non-toxic
61104,316936,0.700000,"""Liberal Democrat,"" my butt.\n\nAfter Maddox's...",0.3,0.2,0.700000,0.7,0.0,0.0,0.0,...,0,0,0,0,0,0.3,4,10,liber democrat butt death obituari former gove...,toxic


In [ ]:
comment_text_ = [i for i in Train_Final_pre_text_data_sample_ML['comment_text'].values]
comment_text_pre_ = [i for i in Train_Final_pre_text_data_sample_ML['comment_text_pre'].values]
target_ = [i for i in Train_Final_pre_text_data_sample_ML['target'].values]
class_ = [i for i in Train_Final_pre_text_data_sample_ML['class'].values]

In [ ]:
Train_Final_pre_text_data_sample_ML

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,comment_text_pre,class
421424,759216,0.000000,A pause in B.C.'s hot real estate market might...,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,1,1,0,0.0,0,4,paus hot real estat market might good thing mi...,non-toxic
1702633,6209669,0.000000,"It's right wing bologna. Colorado, which is ru...",0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,right wing bologna colorado run big oil ga wor...,non-toxic
1022921,5368046,0.000000,"@Osgoodschlotter: That ""dindu"" jab originated ...",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,4,4,jab origin propaganda machin use tool unwittin...,non-toxic
187815,470870,0.600000,The Chinese do not make mistakes when it comes...,0.0,0.7,0.100000,0.1,0.0,0.4,0.0,...,0,0,0,0,0,0.6,10,10,chines not make mistak come protocol forc obam...,toxic
757837,5048051,0.833333,Can we please have a heterosexual leader.\n\nW...,0.0,0.0,0.833333,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,4,6,pleas heterosexu leader would possibl break no...,toxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825252,5130890,0.000000,I'm all in on this one!,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,one,non-toxic
908050,5230772,0.000000,"Steven said ""Your inevitable inability to prov...",0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,0,1,1,0.0,0,4,steven said inevit inabl provid evid taken adm...,non-toxic
319561,633846,0.000000,"Obviously, your opinion shows your lack of int...",0.0,0.0,0.000000,0.0,0.0,NaN,NaN,...,0,0,0,0,2,0.0,0,4,obvious opinion show lack intim oper experi on...,non-toxic
61104,316936,0.700000,"""Liberal Democrat,"" my butt.\n\nAfter Maddox's...",0.3,0.2,0.700000,0.7,0.0,0.0,0.0,...,0,0,0,0,0,0.3,4,10,liber democrat butt death obituari former gove...,toxic


In [ ]:
for i in range(5):
    print('\ncomment_text_ :\n',comment_text_[i])
    print('___'*30,'\n')
    
    print('\ncomment_text_pre_ :\n',comment_text_pre_[i])
    print('___'*30,'\n')
    
    print('\ntarget_ :',target_[i])
    print('___'*30,'\n')
    
    print('\nclass_ :',class_[i],'\n\n')
    print('**'*100,'\n')


comment_text_ :
 A pause in B.C.'s hot real estate market might be a good thing. It might give a chance for more affordable housing to evolve that allows service workers and low income people a chance at accommodations. As it stands now, a lot of Vancouver businesses are understaffed simply because the people they need can't find a place to live nearby.
__________________________________________________________________________________________ 


comment_text_pre_ :
 paus hot real estat market might good thing might give chanc afford hous evolv allow servic worker low incom peopl chanc accommod stand lot vancouv busi simpli peopl need cannot find place live nearbi
__________________________________________________________________________________________ 


target_ : 0.0
__________________________________________________________________________________________ 


class_ : non-toxic 


*******************************************************************************************************

In [ ]:
Train_Final_pre_text_data_ML = preprocessing_ML(Data,submission_data=False)

100%|██████████| 1804874/1804874 [00:01<00:00, 1267895.85it/s]


In [ ]:
Train_Final_pre_text_data_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 47 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   target                               float64
 2   comment_text                         object 
 3   severe_toxicity                      float64
 4   obscene                              float64
 5   identity_attack                      float64
 6   insult                               float64
 7   threat                               float64
 8   asian                                float64
 9   atheist                              float64
 10  bisexual                             float64
 11  black                                float64
 12  buddhist                             float64
 13  christian                            float64
 14  female                               float64
 15  heterosexual                    

Saving File

In [ ]:
Train_Final_pre_text_data_ML.to_csv('Train_Final_pre_text_data_ML_1.csv',index=False)

Load

In [ ]:
Train_Final_pre_text_data_ML = pd.read_csv(r'Train_Final_pre_text_data_ML_1.csv')

In [ ]:
Train_Final_pre_text_data_ML

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,comment_text_pre,class
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,cool like want mother read realli great idea w...,non-toxic
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,thank would make life lot less keep not let an...,non-toxic
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,urgent design problem take impress,non-toxic
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,someth abl instal site releas,non-toxic
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,...,0,0,0,1,0,0.0,4,47,guy bunch loser,toxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804869,6333967,0.000000,"Maybe the tax on ""things"" would be collected w...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,mayb tax thing would collect product import re...,non-toxic
1804870,6333969,0.000000,What do you call people who STILL think the di...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,call peopl still think divin role creation,non-toxic
1804871,6333982,0.000000,"thank you ,,,right or wrong,,, i am following ...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,thank right wrong follow advic,non-toxic
1804872,6334009,0.621212,Anyone who is quoted as having the following e...,0.030303,0.030303,0.045455,0.621212,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,66,anyon quot follow exchang even would receiv vo...,toxic


In [ ]:
Train_Final_pre_text_data_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 47 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   target                               float64
 2   comment_text                         object 
 3   severe_toxicity                      float64
 4   obscene                              float64
 5   identity_attack                      float64
 6   insult                               float64
 7   threat                               float64
 8   asian                                float64
 9   atheist                              float64
 10  bisexual                             float64
 11  black                                float64
 12  buddhist                             float64
 13  christian                            float64
 14  female                               float64
 15  heterosexual                    

In [ ]:
Train_Final_pre_text_data_ML['comment_text_pre'] = Train_Final_pre_text_data_ML['comment_text_pre'].astype('string')

In [ ]:
Train_Final_pre_text_data_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 47 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   target                               float64
 2   comment_text                         object 
 3   severe_toxicity                      float64
 4   obscene                              float64
 5   identity_attack                      float64
 6   insult                               float64
 7   threat                               float64
 8   asian                                float64
 9   atheist                              float64
 10  bisexual                             float64
 11  black                                float64
 12  buddhist                             float64
 13  christian                            float64
 14  female                               float64
 15  heterosexual                    

In [ ]:
#Test Data

In [ ]:
Test_data =  pd.read_csv('/content/test.csv')

In [ ]:
Test_Final_pre_text_data_ML = preprocessing_ML(Test_data,submission_data=True)

100%|██████████| 97320/97320 [00:00<00:00, 117321.17it/s]


In [ ]:
Test_Final_pre_text_data_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97320 entries, 0 to 97319
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                97320 non-null  int64 
 1   comment_text      97320 non-null  object
 2   comment_text_pre  97320 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [ ]:
#Saving File
Test_Final_pre_text_data_ML.to_csv('Test_Final_pre_text_data_ML_1.csv',index=False)

In [ ]:
#Loading file
Test_Final_pre_text_data_ML = pd.read_csv(r'Test_Final_pre_text_data_ML_1.csv')

In [ ]:
Test_Final_pre_text_data_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97320 entries, 0 to 97319
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                97320 non-null  int64 
 1   comment_text      97320 non-null  object
 2   comment_text_pre  96678 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


Converting comment_text_pre from object data type to string data type

In [ ]:
Test_Final_pre_text_data_ML['comment_text_pre'] = Test_Final_pre_text_data_ML['comment_text_pre'].astype('string')

In [ ]:
Test_Final_pre_text_data_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97320 entries, 0 to 97319
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                97320 non-null  int64 
 1   comment_text      97320 non-null  object
 2   comment_text_pre  96678 non-null  string
dtypes: int64(1), object(1), string(1)
memory usage: 2.2+ MB


Preprocessing the data for DL models


In [ ]:
Train_Final_pre_text_data_DL = preprocessing_DL(Data,submission_data=False)

100%|██████████| 1804874/1804874 [00:01<00:00, 1251248.57it/s]


In [ ]:
Train_Final_pre_text_data_DL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 47 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   target                               float64
 2   comment_text                         object 
 3   severe_toxicity                      float64
 4   obscene                              float64
 5   identity_attack                      float64
 6   insult                               float64
 7   threat                               float64
 8   asian                                float64
 9   atheist                              float64
 10  bisexual                             float64
 11  black                                float64
 12  buddhist                             float64
 13  christian                            float64
 14  female                               float64
 15  heterosexual                    

In [ ]:
Train_Final_pre_text_data_DL.to_csv('Train_Final_pre_text_data_DL_1.csv',index=False)

In [ ]:
Train_Final_pre_text_data_DL = pd.read_csv(r'Train_Final_pre_text_data_DL_1.csv')
Train_Final_pre_text_data_DL

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,comment_text_pre,class
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,cool like want mother read really great idea w...,non-toxic
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,thank would make life lot less keep not let an...,non-toxic
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,urgent design problem taking impressive,non-toxic
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,something able install site releasing,non-toxic
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,...,0,0,0,1,0,0.0,4,47,guys bunch losers,toxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804869,6333967,0.000000,"Maybe the tax on ""things"" would be collected w...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,maybe tax things would collected product impor...,non-toxic
1804870,6333969,0.000000,What do you call people who STILL think the di...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,call people still think divine role creation,non-toxic
1804871,6333982,0.000000,"thank you ,,,right or wrong,,, i am following ...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,4,thank right wrong following advice,non-toxic
1804872,6334009,0.621212,Anyone who is quoted as having the following e...,0.030303,0.030303,0.045455,0.621212,0.0,NaN,NaN,...,0,0,0,0,0,0.0,0,66,anyone quoted following exchange even would re...,toxic


In [ ]:
Train_Final_pre_text_data_DL['comment_text_pre'] = Train_Final_pre_text_data_DL['comment_text_pre'].astype('string')

In [ ]:
Train_Final_pre_text_data_DL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804874 entries, 0 to 1804873
Data columns (total 47 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   target                               float64
 2   comment_text                         object 
 3   severe_toxicity                      float64
 4   obscene                              float64
 5   identity_attack                      float64
 6   insult                               float64
 7   threat                               float64
 8   asian                                float64
 9   atheist                              float64
 10  bisexual                             float64
 11  black                                float64
 12  buddhist                             float64
 13  christian                            float64
 14  female                               float64
 15  heterosexual                    

In [ ]:
Test_data =  pd.read_csv("/content/test.csv")

In [ ]:
Test_Final_pre_text_data_DL = preprocessing_DL(Test_data,submission_data=True)

100%|██████████| 97320/97320 [00:00<00:00, 110860.07it/s]


In [ ]:
Test_Final_pre_text_data_DL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97320 entries, 0 to 97319
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                97320 non-null  int64 
 1   comment_text      97320 non-null  object
 2   comment_text_pre  97320 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [ ]:
#Saving File
Test_Final_pre_text_data_DL.to_csv('Test_Final_pre_text_data_DL_1.csv',index=False)

In [ ]:
#Loding File
Test_Final_pre_text_data_DL = pd.read_csv(r'Test_Final_pre_text_data_DL_1.csv')

In [ ]:
Test_Final_pre_text_data_DL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97320 entries, 0 to 97319
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                97320 non-null  int64 
 1   comment_text      97320 non-null  object
 2   comment_text_pre  96681 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [ ]:
Test_Final_pre_text_data_DL['comment_text_pre'] = Test_Final_pre_text_data_DL['comment_text_pre'].astype('string')

In [ ]:
Test_Final_pre_text_data_DL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97320 entries, 0 to 97319
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                97320 non-null  int64 
 1   comment_text      97320 non-null  object
 2   comment_text_pre  96681 non-null  string
dtypes: int64(1), object(1), string(1)
memory usage: 2.2+ MB


In [ ]:
#Feature Engineering

##Feature Engineering

All about Data : There are Two data set train.csv and Test.csv
train.csv : we have to break train.csv into 3 part.(do it for all vectorizers)
Train
Test
CV

In [ ]:
Train_Final_pre_text_data_DL['comment_text_pre'].isnull().sum().sum()

12164

In [ ]:
Train_Final_pre_text_data_ML['comment_text_pre'].isnull().sum().sum()

12195

In [ ]:
#Drop nan values

In [ ]:
Train_Final_pre_text_data_ML.dropna(subset = ["comment_text_pre"],inplace=True)

In [ ]:
Train_Final_pre_text_data_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1792679 entries, 0 to 1804873
Data columns (total 47 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   id                                   int64  
 1   target                               float64
 2   comment_text                         object 
 3   severe_toxicity                      float64
 4   obscene                              float64
 5   identity_attack                      float64
 6   insult                               float64
 7   threat                               float64
 8   asian                                float64
 9   atheist                              float64
 10  bisexual                             float64
 11  black                                float64
 12  buddhist                             float64
 13  christian                            float64
 14  female                               float64
 15  heterosexual                    

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Train_Final_pre_text_data_ML['comment_text_pre'].values, Train_Final_pre_text_data_ML['class'].values, test_size=0.15,stratify=Train_Final_pre_text_data_ML['class'].values ,random_state=42)


In [ ]:
x_train, x_cv, y_train, y_cv = train_test_split(x_train,y_train,stratify=y_train, test_size=0.15, random_state=42)


In [ ]:
import pickle
with open('data_train_test_cv_csv.pkl', 'wb') as file:
    pickle.dump([x_train, x_cv , x_test, y_train, y_cv, y_test], file)

In [ ]:
import pickle
  
# Open the file in binary mode
with open('data_train_test_cv_csv.pkl', 'rb') as file:
      
    # Call load method to deserialze
    x_train, x_cv , x_test, y_train, y_cv, y_test = pickle.load(file)
 

#BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_BOW= CountVectorizer(min_df=10, max_features=10000)
vectorizer_BOW.fit(x_train)

x_train_BOW = vectorizer_BOW.transform(x_train)
x_cv_BOW = vectorizer_BOW.transform(x_cv)
x_test_BOW = vectorizer_BOW.transform(x_test)

In [ ]:
len(vectorizer_BOW.get_feature_names())

10000

In [ ]:
print('x_train_BOW:',x_train_BOW.shape)
print('x_cv_BOW:',x_cv_BOW.shape)
print('x_test_BOW:',x_test_BOW.shape)

x_train_BOW: (1295210, 10000)
x_cv_BOW: (228567, 10000)
x_test_BOW: (268902, 10000)


In [ ]:
Test_Final_pre_text_data_ML = pd.read_csv(r'Test_Final_pre_text_data_ML_1.csv')
Test_Final_pre_text_data_ML['comment_text_pre'] = Test_Final_pre_text_data_ML['comment_text_pre'].astype('string')
## Droping NAN values
Test_Final_pre_text_data_ML.dropna(subset = ["comment_text_pre"],inplace=True)
Test_Final_pre_text_data_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96678 entries, 0 to 97319
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                96678 non-null  int64 
 1   comment_text      96678 non-null  object
 2   comment_text_pre  96678 non-null  string
dtypes: int64(1), object(1), string(1)
memory usage: 3.0+ MB


In [ ]:
Test_Final_pre_text_data_ML_BOW = vectorizer_BOW.transform(Test_Final_pre_text_data_ML['comment_text_pre'])

In [ ]:
print('Test_Final_pre_text_data_ML_BOW:',Test_Final_pre_text_data_ML_BOW.shape)

In [ ]:
import pickle
with open('data_BOW.pkl', 'wb') as file:
    pickle.dump([x_train_BOW , x_cv_BOW , x_test_BOW, y_train, y_cv, y_test,Test_Final_pre_text_data_ML_BOW], file)


In [ ]:
import pickle
  
# Open the file in binary mode
with open('data_BOW.pkl', 'rb') as file:
      
    # Call load method to deserialze
    x_train_BOW , x_cv_BOW , x_test_BOW, y_train, y_cv, y_test,Test_Final_pre_text_data_ML_BOW = pickle.load(file)
 